In [ ]:
import pandas as pd
import os
import joblib # Biblioteca para salvar o modelo
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import LabelEncoder

# --- Passo 1: Carregar os Dados Limpos ---
caminho_dados_limpos = os.path.join('..', 'data', 'dados_limpos.csv')

try:
    df = pd.read_csv(caminho_dados_limpos)
    print("Arquivo de dados limpos carregado com sucesso!")
except FileNotFoundError:
    print(f"Erro: Arquivo não encontrado em '{caminho_dados_limpos}'.")

# --- Passo 2: Preparação dos Dados para o Modelo ---

# Nosso objetivo (alvo) é prever 'risco_evasao_declarado'.
# Vamos simplificar para uma classificação binária: 'Sim' vs 'Não'.
# Por enquanto, vamos remover os 'Talvez' para ter um alvo claro.
df_modelo = df[df['risco_evasao_declarado'].isin(['Sim', 'Não'])].copy()

# Selecionar as 'features' (características) que usaremos para treinar o modelo.
# Excluímos colunas de texto livre, IDs ou a resposta original.
features = [
    'curso', 'periodo', 'idade', 'genero', 'cor_raca',
    'renda_familiar', 'trabalha'
]

# A variável que queremos prever (alvo ou target)
target = 'risco_evasao_declarado'

# Separar os dados em X (features) e y (target)
X = df_modelo[features]
y = df_modelo[target]

# Converter todas as colunas categóricas em números usando One-Hot Encoding.
# Isso cria novas colunas para cada categoria (ex: curso_Técnico_Informática, curso_Licenciatura...)
X_encoded = pd.get_dummies(X, drop_first=True)

# Guardar os nomes das colunas usadas no modelo, para usarmos no dashboard depois
model_columns = X_encoded.columns
joblib.dump(model_columns, os.path.join('..', 'dashboard', 'colunas_modelo.joblib'))


# --- Passo 3: Dividir os Dados em Treino e Teste ---
# 80% dos dados para treinar o modelo, 20% para testar sua performance.
X_train, X_test, y_train, y_test = train_test_split(
    X_encoded, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTamanho do conjunto de treino: {X_train.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_test.shape[0]} amostras")


# --- Passo 4: Treinar o Modelo de Machine Learning ---
print("\nIniciando o treinamento do modelo RandomForestClassifier...")

# Inicializa o modelo. random_state=42 garante que o resultado seja sempre o mesmo.
# class_weight='balanced' ajuda o modelo a lidar com classes desbalanceadas (se tivermos mais 'Não' do que 'Sim').
model = RandomForestClassifier(random_state=42, class_weight='balanced')

# Treina o modelo com os dados de treino
model.fit(X_train, y_train)

print("Treinamento concluído!")


# --- Passo 5: Avaliar a Performance do Modelo ---
print("\nAvaliando o modelo com os dados de teste...")

# Faz as previsões nos dados de teste (dados que o modelo nunca viu)
y_pred = model.predict(X_test)

# Compara as previsões com os resultados reais
accuracy = accuracy_score(y_test, y_pred)
print(f"\nAcurácia do modelo: {accuracy:.2f}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))


# --- Passo 6: Salvar o Modelo Treinado ---
# Salva o objeto do modelo treinado em um arquivo para ser usado pelo dashboard.
caminho_modelo = os.path.join('..', 'dashboard', 'modelo_evasao.joblib')
joblib.dump(model, caminho_modelo)

print(f"\nModelo salvo com sucesso em: '{caminho_modelo}'")